In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName("ShobhitApp") \
.getOrCreate()

In [2]:
data=[["1"]]
df=spark.createDataFrame(data,["id"])

from pyspark.sql.functions import *
#current_date() & current_timestamp()
df = df.withColumn("current_date",current_date()) \
  .withColumn("current_timestamp",current_timestamp()) \
.withColumn("last_day_prev_month", last_day(add_months(col("current_date"), -1)))

In [3]:
df.show(1)

+---+------------+--------------------+-------------------+
| id|current_date|   current_timestamp|last_day_prev_month|
+---+------------+--------------------+-------------------+
|  1|  2022-04-21|2022-04-21 01:47:...|         2022-03-31|
+---+------------+--------------------+-------------------+



In [4]:
data=[["00001"],["0011"],["002"]]
df=spark.createDataFrame(data,["id"])
df.withColumn('id_new', regexp_replace('id', r'^[0]*', '')).show()

+-----+------+
|   id|id_new|
+-----+------+
|00001|     1|
| 0011|    11|
|  002|     2|
+-----+------+



##### Extended Plan

In [5]:
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)

dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+-----

In [6]:
res = empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"inner")

In [7]:
res.select('*').explain(extended=True)

== Parsed Logical Plan ==
'Project [*]
+- Join Inner, (cast(emp_dept_id#43 as bigint) = dept_id#76L)
   :- LogicalRDD [emp_id#39L, name#40, superior_emp_id#41L, year_joined#42, emp_dept_id#43, gender#44, salary#45L], false
   +- LogicalRDD [dept_name#75, dept_id#76L], false

== Analyzed Logical Plan ==
emp_id: bigint, name: string, superior_emp_id: bigint, year_joined: string, emp_dept_id: string, gender: string, salary: bigint, dept_name: string, dept_id: bigint
Project [emp_id#39L, name#40, superior_emp_id#41L, year_joined#42, emp_dept_id#43, gender#44, salary#45L, dept_name#75, dept_id#76L]
+- Join Inner, (cast(emp_dept_id#43 as bigint) = dept_id#76L)
   :- LogicalRDD [emp_id#39L, name#40, superior_emp_id#41L, year_joined#42, emp_dept_id#43, gender#44, salary#45L], false
   +- LogicalRDD [dept_name#75, dept_id#76L], false

== Optimized Logical Plan ==
Join Inner, (cast(emp_dept_id#43 as bigint) = dept_id#76L)
:- Filter isnotnull(emp_dept_id#43)
:  +- LogicalRDD [emp_id#39L, name#40,